In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

# Go up one directory from `b/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()

DEFAULT_TRANSFORMERS_MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 16_000,
    "load_in_4bit": True,
    "enable_thinking": False,  # main_gpu interpretation
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **DEFAULT_TRANSFORMERS_MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

╔════════════════════════════════════════════════════════════════════╗
║                                                                    ║
║ ▓▓▓▓▓   ▓▓▓▓                                   ▓▓▓                 ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓                   ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓                  ║
║  ▓▓   ▓▓ ▓▓  ▓     ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓                   ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓                   ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready   ║
║ Faster builds, clearer flows, production-first                     ║
║                                                                    ║
║ Version: unknown | Python: 3.11.13                                 ║
║ OS: Linux 6.14.0-33-generic (x86_64)                               ║
║ Torch: 2.7.1+cu126   CUDA: yes (12.6)                              ║
║ MPS:

/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     | 2025-11-04 13:30:40 | config.py:<module> | PyTorch version 2.7.1+cu126 available.


/home/nomi/anaconda3/envs/LLMs/lib/python3.11/importlib/__init__.py:126: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-04 13:30:41 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-11-04 13:30:43 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-11-04 13:30:43 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.
INFO     | 2025-11-04 13:30:43 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.23it/s]

INFO     | 2025-11-04 13:30:44 | transformers.py:init_model | Transformers model initialized successfully.


In [3]:
from IPython.display import display, Markdown, clear_output

system_prompt = "You are a joker."
user_prompt = """Joke about yourself."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    print(chunk, flush=True, end="")

Ah, you caught me! I'm not just a joker—I'm a *joke* in disguise! 😏

But don't worry, I'm not here to ruin your day. I'm here to make it *funnier*! What's your favorite kind of joke? I've got a whole repertoire—pun-based, wordplay, or just straight-up absurdity. Let me know and I'll deliver! 🎩🎭

## RAG wiring so the agent “understands” the Neurosurf codebase

You’ll ingest the repo once, then run a retriever to answer code questions. The Planner can call the retriever first to form a precise implementation plan.

### FileReader and Chunker Test

In [4]:
# scripts/index_repo_for_rag.py
from neurosurfer.rag.ingestor import RAGIngestor
from neurosurfer.rag.chunker import Chunker
from neurosurfer.rag.filereader import FileReader
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder

embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
vs = ChromaVectorStore(collection_name="neurosurf-repo")
ing = RAGIngestor(
    embedder=embedder, 
    vector_store=vs, 
    chunker=Chunker(), 
    file_reader=FileReader(),
    default_metadata={"collection": "neurosurf"}
)

ing.add_directory("/home/nomi/workspace/neurosurfer/neurosurfer")  # the repo root
print(ing.build())


INFO     | 2025-11-04 13:30:48 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-04 13:30:48 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-11-04 13:30:48 | posthog.py:__init__ | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[Init] ChromaVectorStore initialized with collection: neurosurf-repo


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.84it/s]


{'status': 'ok', 'sources': 92, 'chunks': 607, 'unique_chunks': 607, 'added': 607, 'finished_at': 1762248651.173717}


### Query TIme Helper

In [ ]:
# scripts/rag_helper.py
from neurosurfer.agents.rag import RAGAgent, RAGRetrieveConfig
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
from neurosurfer.models.chat_models.openai import OpenAIModel

vs = ChromaVectorStore(collection_name="neurosurf-repo")
# embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
# llm = OpenAIModel(model_name="gpt-4o-mini")
rag_agent = RAGAgent(
    llm=LLM, 
    vectorstore=vs, 
    embedder=embedder, 
    config=RAGRetrieveConfig(top_k=6)
)

# def plan_with_context(user_goal: str) -> str:
#     res = rag.retrieve(
#         user_query=user_goal,
#         base_system_prompt="You are a senior Neurosurf contributor. Use the context to plan precise code changes.",
#         base_user_prompt="Context:\n{context}\n\nGoal:\n" + user_goal + "\n\nReturn a step-by-step plan."
#     )
#     plan = LLM.ask(
#         user_prompt=res.base_user_prompt.replace("{context}", res.context),
#         system_prompt=res.base_system_prompt,
#         max_new_tokens=res.max_new_tokens,
#         temperature=0.2,
#         stream=False
#     ).choices[0].message.content
#     return plan


[Init] ChromaVectorStore initialized with collection: neurosurf-repo


In [ ]:
from IPython.display import display, Markdown

# test retrieve
# retrieved_obj = rag_agent.retrieve("What is the main difference between transformers and unsloth?")
# print(retrieved_obj.context)

# test run
llm_response = rag_agent.run("What do you think where we can improve the repo?")
response = ""
md_display = display(Markdown(""), display_id=True)
for chunk in llm_response:
    response += chunk
    md_display.update(Markdown(response))


The question "What do you think where we can improve the repo?" is asking for suggestions on how to improve a repository, likely referring to a codebase or project. Based on the context provided, here are some areas where improvements could be made:

### 1. **Code Quality and Structure**
- **Refactoring**: Improve code structure, reduce technical debt, and make the codebase more maintainable.
- **Code Standards**: Enforce consistent coding standards (e.g., PEP8, ESLint, etc.) using linters and formatters.
- **Modularization**: Break down large functions or modules into smaller, reusable components.

### 2. **Testing Coverage**
- **Unit Tests**: Ensure all core functionalities are covered by unit tests.
- **Integration Tests**: Add integration tests to validate interactions between components.
- **Test Coverage Tools**: Use tools like `coverage.py` or `pytest-cov` to measure and improve test coverage.

### 3. **Documentation**
- **API Documentation**: Improve documentation for APIs, especially for external users or contributors.
- **README and Setup Instructions**: Make setup and contribution instructions clearer and more detailed.
- **Comments in Code**: Add meaningful comments to complex logic or non-obvious parts of the code.

### 4. **CI/CD and Automation**
- **CI/CD Pipeline**: Set up a CI/CD pipeline (e.g., GitHub Actions, GitLab CI) to automate testing, linting, and deployment.
- **Code Reviews**: Enforce pull request reviews to ensure code quality before merging.
- **Automated Linting**: Integrate linters and formatters into the CI/CD pipeline to catch issues early.

### 5. **Dependency Management**
- **Update Dependencies**: Regularly update dependencies to ensure security and performance.
- **Dependency Locking**: Use tools like `pipenv`, `poetry`, or `yarn` to lock dependencies and avoid version conflicts.

### 6. **Performance Optimization**
- **Profiling**: Use profiling tools to identify performance bottlenecks.
- **Caching**: Implement caching for frequently accessed data or expensive computations.
- **Asynchronous Processing**: Use asynchronous programming where appropriate to improve responsiveness.

### 7. **Error Handling and Logging**
- **Robust Error Handling**: Improve error handling to make the application more resilient.
- **Logging**: Add detailed logging to help with debugging and monitoring.

### 8. **Security**
- **Security Audits**: Regularly perform security audits and vulnerability scans.
- **Input Validation**: Ensure all user inputs are properly validated to prevent injection attacks.
- **Authentication and Authorization**: Implement proper access controls and authentication mechanisms.

### 9. **User Experience (UX)**
- **CLI/UI Improvements**: If the repo includes a CLI or web interface, improve the user experience.
- **Error Messages**: Provide clear and helpful error messages to users.

### 10. **Community and Collaboration**
- **Onboarding for New Contributors**: Improve onboarding documentation for new contributors.
- **Issue Management**: Use issue tracking tools (e.g., GitHub Issues, Jira) to manage bugs and feature requests.
- **Contributor Guidelines**: Provide clear guidelines for contributing to the project.

### 11. **Code Formatting and Style**
- **Consistent Formatting**: Use tools like `black`, `isort`, or `prettier` to enforce consistent code formatting.
- **Code Style Guides**: Document the preferred code style and enforce it through CI.

### 12. **Performance Monitoring**
- **Monitoring Tools**: Integrate performance monitoring tools (e.g., Prometheus, New Relic) to track application performance in production.

### 13. **Testing for Edge Cases**
- **Edge Case Testing**: Ensure the code handles edge cases and unexpected inputs gracefully.

### 14. **Code Reusability**
- **Shared Libraries**: Extract common functionality into shared libraries or utility modules.
- **Avoid Duplication**: Reduce code duplication by using helper functions or utility classes.

### 15. **Version Control Best Practices**
- **Branching Strategy**: Use a consistent branching strategy (e.g., Git Flow, GitHub Flow).
- **Commit Messages**: Improve commit message clarity and consistency.

### 16. **Testing for Different Environments**
- **Environment-Specific Tests**: Ensure tests run in different environments (e.g., development, staging, production).
- **Dockerization**: Containerize the application using Docker for consistent environments.

### 17. **Code Readability**
- **Descriptive Variable Names**: Use clear and descriptive variable and function names.
- **Avoid Magic Numbers**: Replace magic numbers with constants or enums.

### 18. **Documentation for Internal Tools**
- **Internal Tools Documentation**: If the repo includes internal tools or scripts, document them clearly for team members.

### 19. **Performance of External Libraries**
- **Optimize External Libraries**: Replace or optimize external libraries that are known to be slow or inefficient.

### 20. **Code Reviews and Pair Programming**
- **Code Reviews**: Encourage regular code reviews to catch issues early and share knowledge.
- **Pair Programming**: Use pair programming for complex or critical parts of the codebase.

---

### Summary
To improve the repo, focus on **code quality**, **testing**, **documentation**, **automation**, **security**, and **maintainability**. These areas are critical for long-term project success and team productivity. If you have a specific part of the repo in mind, I can provide more targeted suggestions.

Batches: 100%|██████████| 1/1 [00:00<00:00, 33.07it/s]

INFO     | 2025-11-04 14:33:49 | agent.py:retrieve | [RAGRetriever] Retrieved 6 chunks


## Agent bootstrap (Planner/Coder/Tester as ReActAgent)

In [ ]:
from neurosurfer.agents.react import ReActAgent, ReActConfig
from neurosurfer.tools import Toolkit
from neurosurfer.models.chat_models.openai import OpenAIModel
# Tools
from tools.codegen import CodegenTool
from tools.fs_write import WriteFileTool
from tools.pytest_runner import PytestRunTool
from tools.lint import LintTool
from tools.docs_update import DocsUpdateTool
from tools.git_ops import GitOpsTool

SYSTEM_PROMPT = """
You are the Neurosurf Self-Improvement Agent.
Goals:
1) Plan changes to extend Neurosurf (tools/agents/adapters/docs/tests) safely.
2) Use only the registered tools. Validate with lint + pytest.
3) Produce small, reviewable increments. Write docs and tests for each change.
Rules:
- Never write outside the repository.
- Prefer minimal public APIs and follow existing code style.
- After green tests and lint, prepare a commit and short changelog.
"""

def build_toolkit() -> Toolkit:
    tk = Toolkit()
    # for t in [CodegenTool(), WriteFileTool(), PytestRunTool(), LintTool(), DocsUpdateTool(), GitOpsTool()]:
    for t in [CodegenTool(llm=LLM, rag=rag), LintTool()]:
        tk.register_tool(t)
    return tk

def make_agent():
    # llm = OpenAIModel(model_name="gpt-4o-mini")  # or TransformersModel(...)
    # agent = ReActAgent(toolkit=build_toolkit(), llm=LLM, verbose=False, specific_instructions=SYSTEM_PROMPT)
    agent = ReActAgent(
        toolkit=build_toolkit(),
        llm=LLM,
        specific_instructions="Always be concise in your answers.",
        config=ReActConfig(
            temperature=0.6,
            max_new_tokens=4096,
            allow_input_pruning=True,
            repair_with_llm=True,
            skip_special_tokens=True
        )
    )
    return agent


agent = make_agent()
# Example task: scaffold a tool, write tests, run lint/tests, and prepare commit
# TASK = """Add a 'calculator' tool under neurosurf/tools/builtin/calculator.py, 
# write tests in tests/tools/test_calculator.py, add docs in docs/tools/calculator.md,
# ensure ruff/pytest pass, and commit on branch feature/calculator.
# """
TASK = """Generate a tool `weather` which will call external api to get weather data."""

for chunk in agent.run(TASK, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")


INFO     | 2025-11-04 13:30:53 | toolkit.py:register_tool | Registered tool: codegen_tool
INFO     | 2025-11-04 13:30:53 | toolkit.py:register_tool | Registered tool: lint


[🧠] Chain of Thoughts...
Thought: I need to generate a tool called `weather_tool`. I will use the `codegen_tool` to create this tool with the necessary parameters and specifications.

Action: {
  "tool": "codegen_tool",
  "inputs": {
    "name": "weather_tool",
    "module_path": "tools/weather_tool.py",
    "purpose": "Fetches current weather data for a given location.",
    "when_to_use": "Use this tool when you need to retrieve current weather information for a specific location.",
    "inputs": [
      {"name": "location", "type": "str", "description": "The location for which to fetch the weather data.", "required": true}
    ],
    "returns": {"type": "dict", "description": "A dictionary containing weather data such as temperature, humidity, and conditions."}
  },
  "final_answer": true
}

[🔧] Tool: codegen_tool
[📤] Inputs: {'name': 'weather_tool', 'module_path': 'weather_tool.py', 'purpose': 'Fetch weather data for a given 
location', 'when_to_use': 'When you need to retrieve current weather information', 'inputs': [{'name': 'location', 
'type': 'string', 'description': 'The location to check the weather for', 'required': True}], 'returns': {'type': 
'dict', 'description': 'Dictionary containing weather data'}}

Batches: 100%|██████████| 1/1 [00:00<00:00, 54.84it/s]

INFO     | 2025-11-04 13:31:01 | rag_retriever_agent.py:retrieve | [RAGRetriever] Retrieved 6 chunks


Observation: Generated and wrote: weather_tool.py



[🧠] Chain of Thoughts...
Final Answer: The `weather_tool` has been successfully generated and saved as `tools/weather_tool.py`. It fetches current weather data for a given location and returns a dictionary containing weather details such as temperature, humidity, and conditions.

### RAG Ingester Test

In [6]:
# Initialize Embedder model
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder

# https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#llama_cpp.Llama
HUGGINGFACE_EMBEDDING_MODEL: str = '/home/nomi/workspace/Model_Weights/e5-large-v2'

EMBEDDER = SentenceTransformerEmbedder(model_name=HUGGINGFACE_EMBEDDING_MODEL)

In [7]:
# !pip install chromadb==0.6.3

In [8]:
# Initialize vector Store
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.vectorstores.base import Doc

collection_name = "neurosurfer_collection_1"
vectorstore = ChromaVectorStore(collection_name=collection_name, persist_directory="chroma_storage")
vectorstore.clear_collection()

text = "This is a test document"
embedding = EMBEDDER.embed(text)
doc = Doc(
    id="1234567890",
    text=text,
    embedding=embedding,
    metadata=None
)
vectorstore.add_documents(docs=[doc])
vectorstore.list_all_documents()[0]

[Init] ChromaVectorStore initialized with collection: neurosurfer_collection_1


Doc(id='2bd59215-e714-4ac6-8b37-9876254ad48f', text='This is a test document', embedding=array([-0.01322937, -0.04702759,  0.05621338, ..., -0.01470184,
        0.03540039,  0.03302002]), metadata=None)

In [9]:
# wire_up.py
import logging
from neurosurfer.rag.ingestor import RAGIngestor

vectorstore.clear_collection()
vectorstore.list_all_documents()

logger = logging.getLogger("rag")
logger.setLevel(logging.INFO)

def progress(event):
    stage = event.get("stage")
    logger.info(f"RAG progress: {event}")

ingestor = RAGIngestor(
    embedder=EMBEDDER,
    vector_store=vectorstore,
    file_reader=filereader,
    chunker=chunker,
    logger=logger,
    progress_cb=progress,
    batch_size=32,
) 

# Add sources
# ingestor.add_directory("/path/to/project", include_exts={".py", ".md", ".json"})
# ingestor.add_texts(["custom context block A", "custom context block B"], base_id="manual")

# file_path = "tmp/s13640-021-00548-4.pdf"
# ingestor.add_files(paths=[file_path])

# file_path = "tmp/FRWrapper.zip"
# file_path = "tmp/MOIPatrol.zip"
# file_path = "tmp/airport-venator.zip"
file_path = "tmp/MOIAuth.zip"
ingestor.add_zipfile(zip_path=file_path)


# # Build index
summary = ingestor.build()
print("Summary:", summary)

print("Total Number of Chunks:", vectorstore.count())

INFO:rag:RAG progress: {'stage': 'start', 'queued_sources': 412}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 1, 'total_sources': 412, 'progress_pct': 0}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 2, 'total_sources': 412, 'progress_pct': 0}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 3, 'total_sources': 412, 'progress_pct': 0}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 4, 'total_sources': 412, 'progress_pct': 0}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 5, 'total_sources': 412, 'progress_pct': 1}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 6, 'total_sources': 412, 'progress_pct': 1}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 7, 'total_sources': 412, 'progress_pct': 1}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed_sources': 8, 'total_sources': 412, 'progress_pct': 1}
INFO:rag:RAG progress: {'stage': 'chunking', 'completed

/home/nomi/workspace/LLM-Powered-Apps/sql_agent/fastapi/tmp/rag_zip_kkh8y1cg


INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 32, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 64, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 96, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 128, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 160, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 192, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 224, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 256, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 288, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 320, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 352, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 384, 'total': 1184}
INFO:rag:RAG progress: {'stage': 'embedding', 'embedded': 416, 'tot

Summary: {'status': 'ok', 'sources': 412, 'chunks': 1279, 'unique_chunks': 1184, 'added': 1184, 'finished_at': 1757314779.7862504}
Total Number of Chunks: 1184


In [10]:
# for doc in vectorstore.similarity_search(
#     query_embedding=EMBEDDER.embed("What is the purpose of the project?"),
#     top_k=2,
# ):
#     print(doc[0].metadata)
#     # print("\n\n")

In [11]:
from neurosurfer.utils.helper import generate_folder_structure
from neurosurfer.rag.constants import supported_file_types

folder_structure = generate_folder_structure(
    root_path=file_path, supported_files=supported_file_types    
)

print(folder_structure)

/
└── MOIAuth
    ├── AuthCentral.API
    │   ├── Applications
    │   │   ├── Application
    │   │   │   ├── CreateApplication
    │   │   │   │   └── CreateApplicationCommand.cs
    │   │   │   ├── DeleteApplication
    │   │   │   │   └── DeleteApplicationCommand.cs
    │   │   │   ├── GetApplication
    │   │   │   │   └── GetApplicationCommand.cs
    │   │   │   ├── GetApplications
    │   │   │   │   └── GetApplicationsCommand.cs
    │   │   │   └── UpdateApplication
    │   │   │       └── UpdateApplicationCommand.cs
    │   │   ├── Client
    │   │   │   ├── AssignApplication
    │   │   │   │   └── AssignApplicationCommand.cs
    │   │   │   ├── ClientApplicationsDto.cs
    │   │   │   ├── ClientDto.cs
    │   │   │   ├── CreateClient
    │   │   │   │   └── CreateClientCommand.cs
    │   │   │   ├── DeleteClient
    │   │   │   │   └── DeleteClientCommand.cs
    │   │   │   ├── GetAssignedApplications
    │   │   │   │   └── GetAssignedApplicationsCommand.cs
    │   │   │   

## Agents

In [12]:
from neurosurfer.agents.rag_retriever_agent import RAGRetrieverAgent

rag_retriever_agent = RAGRetrieverAgent(
    llm=LLM,
    embedder=EMBEDDER,
    vectorstore=vectorstore
)

system_prompt = "You are a helpful assistant."
user_prompt = "Answer the user questions below:\n\n{query}"

retrieved = rag_retriever_agent.retrieve(
    user_query="What is the purpose?",
    base_system_prompt=system_prompt,
    base_user_prompt=user_prompt,
)

In [15]:
from typing import List
# from neurosurfer.agents.docs_generator_agent import DocsGenerator
from neurosurfer.tools.rag.docs_generator import DocsGenerator

# default_sections: List[str] = [
#     "Project Overview",
#     "Project Scope",
#     "System Components and Design",
#     "APIs/Endpoints",
#     "Use Cases",
#     "Installation / Usage / Implementation Guide",
#     "Configuration",
# ]

doc_generator = DocsGenerator(
    llm=LLM,
    rag_retriever_agent=rag_retriever_agent,
    folder_structure=folder_structure,
    n_files_per_section=25,
    candidate_pool_size=500,
    file_key="filename",
)

In [16]:
# plan documentation first
# user_query = "What is the prurpose of the uploaded code?"
# user_query = (
#     "Describe appsettings.json file, keys/values from the code. Put description for each key in the table. Give the output as markdown table.\n"
#     "Generate a mermaid script that will describe API external dependencies (e.g. APIs and databases).\n"
#     "Describe the connectivity between the dependencies.\n"
#     "Deployment step guide for ready-dockerized composed deployment.\n"
#     "Describe APIs and request/response.\n"
# )


# user_query = """Describe appsettings.json onyl for venator-api-appsettings.json. Explain their keys/values from the code. 
# Put description for each key in the table. Give the output as markdown table only, limit explaining config outside the table.\n
# """ 

user_query = """Describe APIs, their request and response structure. Don't include extra sections, only the core sections.
Return only with sections: API Overview, Request Structure, Response Structure, Endpoints, Error Handling, and Authentication and Authorization
"""


# user_query = "generate a detailed readme.md file intended for Github repo. Do not include references."

doc_plan = doc_generator.plan_docs(
    user_query=user_query,
    folder_structure=folder_structure,
    verbose=True
)

{
  "sections": [
    {
      "title": "API Overview",
      "instructions": "Provide a high-level summary of the API, including its purpose, key endpoints, and the overall architecture. Describe the main functionalities and how they relate to the application's business logic. Highlight the main components and their interactions."
    },
    {
      "title": "Request Structure",
      "instructions": "Detail the structure of API requests, including the HTTP methods (GET, POST, PUT, DELETE), headers, and request body formats. Explain the parameters, query strings, and request payloads for each endpoint. Include examples of request formats for common operations."
    },
    {
      "title": "Response Structure",
      "instructions": "Describe the structure of API responses, including HTTP status codes, response headers, and the body format. Explain the meaning of each status code and the structure of the response body for success and error cases. Provide examples of response formats for

In [17]:
import traceback
from neurosurfer.tools.base_tool import ToolResponse

# generate documents
max_total_chunks = 30   # number of chunks to retrieve
try:
    response: ToolResponse = doc_generator(
        user_query=user_query, documentation_plan=doc_plan, max_total_chunks=max_total_chunks
    )
    response = response.observation
except:
    print(traceback.format_exc())

complete_documentation_markdown = ""
for section_chunk in response:
    if isinstance(section_chunk, dict):
        section_chunk = section_chunk["choices"][0]["message"]["content"]
    complete_documentation_markdown += section_chunk.replace("```markdown", "").replace("```", "")
    print(section_chunk, end="", flush=True)


# API Overview

The API serves as the primary interface for managing authentication, authorization, and related business logic within the system. It provides endpoints for handling applications, clients, modules, permissions, roles, users, and more, all of which are integral to the application's core functionality.

## Purpose

The API is designed to support the authentication and authorization workflows of the system, enabling the creation, retrieval, updating, and deletion of various entities such as applications, clients, modules, permissions, roles, and users. It also supports authentication and authorization operations like login, registration, refresh token, and password reset.

## Key Endpoints

- **Applications**: Manage application entities, including create, read, update, and delete operations.
- **Clients**: Handle client-related operations, including assigning applications, getting client details, and managing client lists.
- **Modules**: Provide endpoints for managing modu

In [93]:
# print(complete_documentation_markdown)

In [23]:
from neurosurfer.agents.tools_router_agent import ToolsRouterAgent
from neurosurfer.tools import Toolkit
# from neurosurfer.tools.sql import (
#     RelevantTableSchemaFinderLLM,
#     SQLExecutor,
#     SQLQueryGenerator,
#     FinalAnswerFormatter,
#     DBInsightsTool
# )

from neurosurfer.tools.rag.simple_query_assitant import RagSimpleAnswerTool
from neurosurfer.tools.common.general_query_assistant import GeneralQueryAssistantTool
from neurosurfer.tools.rag.docs_generator import DocsGenerator

toolkit = Toolkit()
toolkit.register_tool(RagSimpleAnswerTool(llm=LLM, rag_retriever_agent=rag_retriever_agent))
toolkit.register_tool(GeneralQueryAssistantTool(llm=LLM))
toolkit.register_tool(
    DocsGenerator(
        llm=LLM,
        rag_retriever_agent=rag_retriever_agent,
        folder_structure=folder_structure,
        n_files_per_section=10,
        candidate_pool_size=500,
        file_key="filename",
        max_total_chunks=30
    )
)

router_agent = ToolsRouterAgent(llm=LLM, toolkit=toolkit)

In [25]:
# print(toolkit.get_tools_description())

In [28]:
print(router_agent._tools_router_system_prompt())

You are a stateless tool router. Given the user's message and the catalog of tools below, output a SINGLE JSON object with EXACTLY these two keys:
{"tool": "<tool_name>", "inputs": {<param>:<value>}, "optimized_query": "<optimized and detailed query clearly explaining the user's message>"}

STRICT RULES:
- Output MUST be valid JSON, on one line, with only the keys "tool" and "inputs".
- Do NOT include any other keys or text (no explanations, no code fences, no thoughts).
- Choose at most one tool.
- Use only explicit inputs defined in that tool's description. Do NOT invent parameters or values.
- Include only required parameters. Do not include optional parameters.
- The optimized value must clearly capture the meaning of the user’s request.  
- If the user asked multiple questions, break them down into structured text (paragraphs or bullet points) rather than a single run-on query.  
- If no tool is appropriate OR required inputs are missing/ambiguous, output: {"tool":"none","inputs":

In [29]:
# user_query = "What is the prurpose of the uploaded code?"
# user_query = (
#     "Describe appsettings.json file, keys/values from the code. Put description for each key in the table. Give the output as markdown table.\n"
#     "Generate a mermaid script that will describe API external dependencies (e.g. APIs and databases).\n"
#     "Describe the connectivity between the dependencies.\n"
#     "Deployment step guide for ready-dockerized composed deployment.\n"
#     "Describe APIs and request/response.\n"
# )

# user_query = (
#     "Describe appsettings.json file, keys/values from the code. Put description for each key in the table. Give the output as markdown table. Return only one section.\n"
#     "Give all configuration in one table. Better to specify seperate column for what the configuration is about? e.g. database, auth etc."
# )

user_query = "What is the weather today?"

# print(user_query)

tool_name, inputs = router_agent._route(user_query=user_query)
print(tool_name)
print(inputs)

general_query_assistant
{'query': 'What is the weather today?', 'user_query': 'The user is asking for the current weather conditions.'}


In [30]:
final_respone = ""
response = router_agent.run(user_query=user_query)
for chunk in response:
    chunk = chunk["choices"][0]["message"]["content"]
    final_respone += chunk
    print(chunk, end="", flush=True)

Using tool:  general_query_assistant
Tool inputs:  {'query': 'What is the weather today?', 'user_query': 'The user is asking for the current weather conditions.'} 


I cannot provide real-time weather information. Please check a reliable weather service or app for the latest updates.

In [16]:
# ingestor.search("How can we initialize sloth model?", top_k=1)

# SQL Agent Test

In [13]:
# vectorstore and db_engine
import os
from neurosurfer.db import SQLDatabase
from dotenv import load_dotenv
load_dotenv()

DB_SERVER: str = os.getenv("db_server", None)
DB_DATABASE: str = os.getenv("db_name", None)
DB_USERNAME: str = os.getenv("db_username", None)
DB_PASSWORD: str = os.getenv("db_password", None)
DB_DRIVER: str = os.getenv("db_driver", None)
DB_PORT: str = os.getenv("port", None)

driver = "mssql+pyodbc"
db_uri = SQLDatabase.build_connection_string(
    server=DB_SERVER,
    database=DB_DATABASE,
    username=DB_USERNAME,
    password=DB_PASSWORD,
    odbc_driver=DB_DRIVER,
    port=DB_PORT,
    driver=driver
)
# print(db_uri)

In [14]:
from neurosurfer.agents.sql_agent import SQLAgent

sql_agent = SQLAgent(
    llm=LLM,
    db_uri=db_uri,
    storage_path=None,
    logger=LOGGER,
    verbose=True
)

In [15]:
sql_agent.sql_schema_store.db.get_usable_table_names()

['AccessRequestComments',
 'AccessRequestDetails',
 'AccessRequestTypes',
 'AccessRequests',
 'Banks',
 'BlackListedPersons',
 'Buildings',
 'Cameras',
 'Cars',
 'CarsEntryLogs',
 'Departments',
 'Designations',
 'EIDAPersonBanks',
 'EIDAPersonPortrait',
 'EIDAPersons',
 'EntryLogs',
 'FaceCarMapping',
 'Gates',
 'GeneralDepartments',
 'GhqEmployeeInfoVw_Gate',
 'GroupAccessRequests',
 'Groups',
 'HR_PICS',
 'PersonAccessRequests',
 'PersonPortrait',
 'Persons',
 'PortraitFirB12',
 'PortraitNectrine',
 'Ranks',
 'RefugeeFinancialTransactions',
 'RefugeePortraits',
 'Refugees',
 'RefugeesTransactions',
 'RetentionSettings',
 'SubBuildings',
 'WorkerInstances',
 'Workers',
 't',
 'v']

In [16]:
schemas = sql_agent.sql_schema_store.get_all_table_schemas()

In [17]:
# print(LLM.ask(user_prompt="Hi! ", stream=False)["choices"][0]["message"]["content"])

In [18]:
table = "Persons"
summary = sql_agent.sql_schema_store.summarize_schema__(table, schemas[table])
summary

{'table_name': 'Persons',
 'summary': 'The Persons table stores information about individuals, including their personal details, employment-related data, and relationships to other entities such as departments, designations, and groups. It also tracks active status, modification history, and other attributes relevant to the system.'}

In [19]:
# print("Num Tables in Vectorstore:", len(agent.sql_schema_store.store))

In [20]:
# train - add database schema to the vectordb
# from tqdm import tqdm

# summarize = "short"
# table = "Persons"
# schema = schemas[table]
# # --------- Level 0: no LLM ----------
# if not summarize:
#     content = f"Table: {table}\nSchema:\n{schema}"
#     print(content)

# # --------- Level 1: short LLM summary + schema ----------
# elif summarize == "short":
#     summary = sql_raginator.summarize_schema_short__(table, schema)
#     content = f"Table: {table}\n\nSummary:\n{summary}\n\nSchema:{schema}"
#     print(content)

# elif summarize == "detailed":
#     summary = sql_raginator.summarize_schema_detailed__(table, schema)
#     content = f"Table: {table}\n{summary}"
#     print(content)

summarize = True
for processed_tables in sql_agent.train(summarize=summarize, force=False):
    pass

In [21]:
sql_agent.sql_schema_store.store.get("Persons")

{'table_name': 'Persons',
 'summary': 'The `Persons` table stores information about individuals, including their personal details, employment-related data, and relationships to other entities such as departments, designations, and groups. It serves as a central repository for person-related data in the system.',
 'schema': '\nCREATE TABLE [Persons] (\n\t[Id] INTEGER NOT NULL IDENTITY(1,1), \n\t[Name] NVARCHAR(500) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[RankId] INTEGER NULL, \n\t[PersonNumber] NVARCHAR(200) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[PersonType] INTEGER NOT NULL, \n\t[DepartmentId] INTEGER NULL, \n\t[DesignationId] INTEGER NULL, \n\t[PhoneNumber] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[RegistrantNumber] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[UnifiedNumber] BIGINT NULL, \n\t[BookNumber] NVARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, \n\t[CreatedDateTime] DATETIME NULL DEFAULT (getdate()), \n\t[UpdatedDateT

In [22]:
print(sql_agent.react_system_prompt())


You are a reasoning agent that answers user questions using a set of external tools.

## Goal:
Think step-by-step and use tools when necessary to find the answer. Do not make assumptions about any hidden variables — tools will be injected with the correct dependencies at runtime.

---

## Behavior Guidelines:

- 🔍 Think step-by-step.
- 📚 If you are confused about the user's question, ask for clarification.
- 🛠️ If a tool is needed, stop and call the most relevant tool using only its **explicit inputs**.
- 🚫 Do NOT include any additional parameters. Only pass what the tool requires.
- 🔁 Call only **one tool at a time**. After each `Observation`, re-evaluate your strategy.
- ✅ If a tool is needed, end your message with the Action block.
- 🧠 Refine the query for the tool which best describes the user's question. If an error or observation occurs, use it to refine your next action.
- ❌ Do NOT repeat the same tool call with the same inputs if it failed before.
- 📝 Rewrite the tool inputs w

In [24]:
from IPython.display import display, Markdown, clear_output
import time

"""
Final Answer: Here is a list of hard questions about Persons, Cameras, and Buildings along with their corresponding SQL queries:

1. **Question:** What are the details of all active persons who are not MOI employees and belong to group 1?
   **SQL Query:** 
   ```sql
   SELECT Id, Name, PersonType, DepartmentId, DesignationId, PhoneNumber, UnifiedNumber, BookNumber, CreatedDateTime, UpdatedDateTime, IsModified, IsActive, IsMOIEmployee, GroupId, CompanyName, DossierId, Gender, GeneralDepartmentId, IsWanted 
   FROM Persons 
   WHERE IsActive = 1 AND IsMOIEmployee = 0 AND GroupId = 1 AND IsWanted = 0;
   ```

2. **Question:** What are the details of all cameras that are active and have a specific type and location?
   **SQL Query:** 
   ```sql
   SELECT Id, NameEn, NameAr, RTSPLink, LocationId, Type, SubBuildingId, GateId, ResultsInterval, FaceDetectionConfidence, MaxAllowedDetection, MaxFrameResolution, MaxCroppedFaceResolution, FrameToReturn, AllowMultipleFaces, CallBackUrl, WorkerId, IsKiosk, FaceDetectorReolution, WorkerInstanceId 
   FROM Cameras 
   WHERE IsActive = 1 AND Type = 1 AND LocationId = 1;
   ```

3. **Question:** What are the details of all buildings that are active and have specific names in English and Arabic?
   **SQL Query:** 
   ```sql
   SELECT Id, NameEn, NameAr 
   FROM Buildings 
   WHERE IsActive = 1 AND NameEn LIKE '%Building%' AND NameAr LIKE '%مبنى%';
   ```

These questions and their corresponding SQL queries provide insights into the data stored in the Persons, Cameras, and Buildings tables.
"""


"""
1. **How does the data retention policy in `RetentionSettings` specifically impact the lifecycle management of refugee financial transaction records stored in `RefugeeFinancialTransactions`?**  
   *(Requires understanding of retention rules and their application to sensitive financial data)*

2. **What multi-table relationship exists between physical security logs (`Cameras`, `EntryLogs`) and personnel access requests (`AccessRequests`, `PersonAccessRequests`), and how does this ensure accountability for facility entry?**  
   *(Tests knowledge of security integration and audit trails)*

3. **How do `BlackListedPersons` and `RefugeePortraits` tables interact to enforce identity verification processes, and what role does `FaceCarMapping` play in this workflow?**  
   *(Involves biometric data linkage and access control validation)*

4. **What are the implications of a person's status transitioning from "authorized" to "unknown" in `RetentionSettings` on their associated records in `EIDAPersons` and `RefugeesTransactions`?**  
   *(Requires analysis of cascading policy effects across tables)*

5. **How does the organizational hierarchy defined in `Departments` and `Designations` translate into specific access control permissions for groups in `GroupAccessRequests`, and what potential security risks could arise from misconfigurations?**  
   *(Tests understanding of role-based access and its dependencies)*
   
"""


user_query = "What are the details of all buildings?"

kwargs = {
    "temperature": 0.4,
    "max_new_tokens": 8000
}

response = sql_agent.run(user_query=user_query, **kwargs)
md_display = display(Markdown(""), display_id=True)
final_answer_started = False
buffer = ""
for chunk in response:
    if final_answer_started:
        if chunk in ["<__final_answer__>", "</__final_answer__>"]:
            continue
        buffer += chunk
        if "Final Answer:" in buffer:
            buffer = "# Final Answer\n"
        md_display.update(Markdown(buffer))
    else:
        if chunk == "<__final_answer__>":
            final_answer_started = True
        else:
            print(chunk, end="", flush=True)

# Final Answer
Here are the details of all buildings:

| Id | NameEn  | NameAr | CreatedDateTime        | UpdatedDateTime | IsModified | IsActive |
|----|----------|--------|--------------------------|-----------------|------------|----------|
| 1 | Building A | مبنى أ | 2024-12-19 12:33:07.667000 | None           | False     | True     |

This table shows the information for each building, including its ID, English name, Arabic name, creation date, last update date, whether it has been modified, and its active status.



[🧠] Chain of Thoughts...
Thought: I need to find out the details of all buildings. To do this, I should first identify the relevant tables that contain building information. I'll use the `relevant_table_schema_finder_llm` tool to find the schema of the relevant tables.

Action: {
"tool": "relevant_table_schema_finder_llm",
"inputs": {
"query": "What are the details of all buildings?"
 },
"final_answer": false
}

[🔧] Tool: relevant_table_schema_finder_llm
[📤] Inputs: {'query': 'What are the details of all buildings?', 'final_answer': False}

Observation: I have retrieved schema for the following tables which are relevant to answer the user query:
Buildings



[🧠] Chain of Thoughts...
Thought: The relevant table for building details has been identified. Now, I will use the `sql_executor` tool to retrieve all the details from the `Buildings` table.

Action: {
"tool": "sql_executor",
"inputs": {
"sql_query": "SELECT * FROM Buildings;"
 },
"final_answer": false
}

[🔧] Tool: sql_executor
[📤] Inputs: {'sql_query': 'SELECT * FROM Buildings;', 'final_answer': False}

Observation: I've executed the SQL query and have fetched the results.



[🧠] Chain of Thoughts...
Thought: The SQL query has been executed and the results have been fetched. Now, I will use the `final_answer_formatter` tool to format the results into a user-friendly response.

Action: {
"tool": "final_answer_formatter",
"inputs": {
"user_query": "What are the details of all buildings?"
 },
"final_answer": true
}

[🔧] Tool: final_answer_formatter
[📤] Inputs: {'user_query': 'What are the details of all buildings?', 'final_answer': True}